In [2]:
import abcrpv_alpha as rpv

In [10]:
import pandas as pd
pd.set_option('display.max_colwidth',None)
myresult=rpv.find_two_lsp_from_signature("ttjjjjjj")
display(myresult)
display(myresult['Chain A'].map(eval))

Results saved in  /home/ys/hep/abc-rpv/abctestrun/abcrpv_alpha/results/two_lsp_from_ttjjjjjj_ALL_ALL.csv


,CAT,LSP A,LSP B,Signature A,Signature B,Chain A,Chain B,Signatures,Signature A (ER),Signature B (ER),Signatures (ER)
129,U D D,t_L,t_L,tjjj,tjjj,"['t_L -- (t) -- B -- (j) -- u -- [j,j]', 't_L -- (t) -- B -- (j) -- d -- [j,j]', 't_L -- (t) -- B -- (j) -- d -- [j,j]']","['t_L -- (t) -- B -- (j) -- u -- [j,j]', 't_L -- (t) -- B -- (j) -- d -- [j,j]', 't_L -- (t) -- B -- (j) -- d -- [j,j]']",ttjjjjjj,3j_l + 1t,3j_l + 1t,6j_l + 2t
130,U D D,t,t,tjjj,tjjj,"['t -- (t) -- B -- (j) -- u -- [j,j]', 't -- (t) -- B -- (j) -- d -- [j,j]', 't -- (t) -- B -- (j) -- d -- [j,j]']","['t -- (t) -- B -- (j) -- u -- [j,j]', 't -- (t) -- B -- (j) -- d -- [j,j]', 't -- (t) -- B -- (j) -- d -- [j,j]']",ttjjjjjj,3j_l + 1t,3j_l + 1t,6j_l + 2t
812,U_3 D D,q,q,tjjj,tjjj,"['q -- (j) -- B -- (t) -- t -- [j,j]', 'q -- (j) -- B -- (j) -- d -- [t,j]', 'q -- (j) -- B -- (j) -- d -- [t,j]']","['q -- (j) -- B -- (t) -- t -- [j,j]', 'q -- (j) -- B -- (j) -- d -- [t,j]', 'q -- (j) -- B -- (j) -- d -- [t,j]']",ttjjjjjj,3j_l + 1t,3j_l + 1t,6j_l + 2t
813,U_3 D D,d,d,tjjj,tjjj,"['d -- (j) -- B -- (t) -- t -- [j,j]']","['d -- (j) -- B -- (t) -- t -- [j,j]']",ttjjjjjj,3j_l + 1t,3j_l + 1t,6j_l + 2t
817,U_3 D D,u,u,tjjj,tjjj,"['u -- (j) -- B -- (t) -- t -- [j,j]', 'u -- (j) -- B -- (j) -- d -- [t,j]', 'u -- (j) -- B -- (j) -- d -- [t,j]']","['u -- (j) -- B -- (t) -- t -- [j,j]', 'u -- (j) -- B -- (j) -- d -- [t,j]', 'u -- (j) -- B -- (j) -- d -- [t,j]']",ttjjjjjj,3j_l + 1t,3j_l + 1t,6j_l + 2t


129    [t_L -- (t) -- B -- (j) -- u -- [j,j], t_L -- (t) -- B -- (j) -- d -- [j,j], t_L -- (t) -- B -- (j) -- d -- [j,j]]
130          [t -- (t) -- B -- (j) -- u -- [j,j], t -- (t) -- B -- (j) -- d -- [j,j], t -- (t) -- B -- (j) -- d -- [j,j]]
812          [q -- (j) -- B -- (t) -- t -- [j,j], q -- (j) -- B -- (j) -- d -- [t,j], q -- (j) -- B -- (j) -- d -- [t,j]]
813                                                                                  [d -- (j) -- B -- (t) -- t -- [j,j]]
817          [u -- (j) -- B -- (t) -- t -- [j,j], u -- (j) -- B -- (j) -- d -- [t,j], u -- (j) -- B -- (j) -- d -- [t,j]]
Name: Chain A, dtype: object

In [11]:
for i in myresult['Chain A'].map(eval).loc[129]:
    print(i)

t_L -- (t) -- B -- (j) -- u -- [j,j]
t_L -- (t) -- B -- (j) -- d -- [j,j]
t_L -- (t) -- B -- (j) -- d -- [j,j]


In [4]:
import os 
import warnings
import pandas as pd

abcrpv_package_path = "/home/ys/hep/abc-rpv/abctestrun/abcrpv_alpha"

In [7]:

def find_one_lsp_from_signature(signature,rpv_coup="ALL",category="ALL",filename="",inclusive=False,ninclusivej=10,ninclusivel=10,ninclusivemet=True,save_results=True,verbose=True):
    assert type(signature) == str, "input signature needs to be str, only taking one signature at a time"
    assert rpv.rmisc.check_signature_format(signature), "Check signature format, allowed syntax:{x}".format(x=rpv.rdef.FINAL_STATE)
    input_signature = signature
    rpv_coup = rpv_coup.upper()
    category = category.upper()
    if category != "ALL":
        if category.count(" ") < 2:
            if category.count("E") > 0:
                raise NameError("Look up rpv.rdef.CAT_DICT for categories' syntax, it seems like youre trying LLE. Try:\n{x}".format(x=rpv.rdef.CAT_DICT["LLE"]))
            if category.count("Q") > 0:
                raise NameError("Look up rpv.rdef.CAT_DICT for categories' syntax, it seems like youre trying LQD. Try:\n{x}".format(x=rpv.rdef.CAT_DICT["LQD"]))
            if category.count("U") > 0:
                raise NameError("Look up rpv.rdef.CAT_DICT for categories' syntax, it seems like youre trying UDD. Try:\n{x}".format(x=rpv.rdef.CAT_DICT["UDD"]))
        if  rpv_coup != "ALL" and category not in rpv.rdef.CAT_DICT[rpv_coup]:
            raise NameError("\""+category+"\" not a category in "+rpv_coup)
    
    if filename == "":
        filename = "one_lsp_that_decays_to_"+signature+"_"+rpv_coup+"_"+category.replace(" ","")+".csv"

    if save_results ==True:
        if ".csv" not in filename:
            filename=filename+".csv"
        results_path = os.path.join(abcrpv_package_path,"results/"+filename)


    rpv_coup = rpv_coup.replace(" ","")
    if category != "ALL" and rpv_coup == "ALL":
        for i,j in rpv.rdef.CAT_DICT.items():
            if category in j:        
                rpv_coup = i

    warnings.filterwarnings("ignore") #ignore panda.append deprecategorye warning, @TODO update this, do this without append
    signature = rpv.rmisc.signature_ordering(signature)
    output_pd = pd.DataFrame()
        

    if "L" in signature or "J"in signature or "3"in signature  :
        signature = (rpv.set_elements_simplified(signature))
        if verbose == True:
            print("More than one signature in input :\n",input_signature)
            print("Will be looking up               :\n",signature)

        if inclusive==True:
            isignatureJ_dat = []
            isignatureL_dat = []
            isignatureL_dat = []
            for isignature in signature:
                isignatureJ_dat = isignatureJ_dat +(rpv.set_elements_simplified(isignature+"J"))
                isignatureL_dat = isignatureL_dat +(rpv.set_elements_simplified(isignature+"L"))
            signature = isignatureJ_dat + isignatureL_dat + signature
            if verbose == True:
                print("Inclusive option chose :\n")
                print("Also Will be looking up {x}+{n}              :\n".format(x=input_signature,n=ninclusivej),isignatureJ_dat)
                print("Also Will be looking up {x}+{n}              :\n".format(x=input_signature,n=ninclusivel),isignatureL_dat)
                print("Also Will be looking up {x}+{n}              :\n".format(x=input_signature,n=ninclusivel),isignatureL_dat)
                ninclusivej
                ninclusivel
                ninclusivemet
                print("Total               :\n",signature)
                

        
    else:
        signature = [signature]

    if rpv_coup == "ALL":
        for _,LSP_dec_df in rpv.ONE_LSP_RPV_DECAY_DICT.items():
            for k in signature:
                output_pd = output_pd.append(LSP_dec_df.loc[(LSP_dec_df['Signatures'] == k)])

    elif rpv_coup != "ALL":
        LSP_dec_df = rpv.ONE_LSP_RPV_DECAY_DICT[rpv_coup.upper()]
        if category == "ALL" :
            for k in signature:
                output_pd = output_pd.append(LSP_dec_df.loc[(LSP_dec_df['Signatures'] == k)])

        elif  category != "ALL" :
            for k in signature:
                output_pd = output_pd.append(LSP_dec_df.loc[(LSP_dec_df['Signatures'] == k) & (LSP_dec_df['Category'] == category)])


    if save_results==True:
        if verbose == True:
            print("Results saved in ",results_path)
        output_pd.to_csv(results_path,index=False)     
    return  output_pd  

In [8]:
find_one_lsp_from_signature("JJL",inclusive=True)

More than one signature in input

Will be looking up               :
 ['jjl', 'jjT', 'tjl', 'tjT', 'bjl', 'bjT', 'tjl', 'tjT', 'ttl', 'ttT', 'tbl', 'tbT', 'bjl', 'bjT', 'tbl', 'tbT', 'bbl', 'bbT']
Inclusive option chose :

Also Will be looking up               :
 ['jjjl', 'tjjl', 'bjjl', 'jjjT', 'tjjT', 'bjjT', 'tjjl', 'ttjl', 'tbjl', 'tjjT', 'ttjT', 'tbjT', 'bjjl', 'tbjl', 'bbjl', 'bjjT', 'tbjT', 'bbjT', 'tjjl', 'ttjl', 'tbjl', 'tjjT', 'ttjT', 'tbjT', 'ttjl', 'tttl', 'ttbl', 'ttjT', 'tttT', 'ttbT', 'tbjl', 'ttbl', 'tbbl', 'tbjT', 'ttbT', 'tbbT', 'bjjl', 'tbjl', 'bbjl', 'bjjT', 'tbjT', 'bbjT', 'tbjl', 'ttbl', 'tbbl', 'tbjT', 'ttbT', 'tbbT', 'bbjl', 'tbbl', 'bbbl', 'bbjT', 'tbbT', 'bbbT']
Also Will be looking up               :
 ['jjll', 'jjTl', 'jjTl', 'jjTT', 'tjll', 'tjTl', 'tjTl', 'tjTT', 'bjll', 'bjTl', 'bjTl', 'bjTT', 'tjll', 'tjTl', 'tjTl', 'tjTT', 'ttll', 'ttTl', 'ttTl', 'ttTT', 'tbll', 'tbTl', 'tbTl', 'tbTT', 'bjll', 'bjTl', 'bjTl', 'bjTT', 'tbll', 'tbTl', 'tbTl', 'tbTT', 'bbll

,Category,LSP,decays via,Signatures,Chain,NV_cascade,Signatures (ER)
481,L Q D,q,l,jjjl,"q -- (j) -- B -- (l) -- l -- [j,j]",2,3j_l + 1l
483,L Q D,q,d,jjjl,"q -- (j) -- B -- (j) -- d -- [l,j]",2,3j_l + 1l
485,L Q D,q,nu,jjjl,"q -- (j) -- W^+ -- (l) -- nu -- [j,j]",2,3j_l + 1l
552,L Q D,d,l,jjjl,"d -- (j) -- B -- (l) -- l -- [j,j]",2,3j_l + 1l
553,L Q D,d,q,jjjl,"d -- (j) -- B -- (j) -- q -- [l,j]",2,3j_l + 1l
...,...,...,...,...,...,...,...
421,U_3 D_3 D_3,tau_L,b,tbbT,"tau_L -- (T) -- B -- (b) -- b -- [t,b]",2,1t + 2b + 1tau
422,U_3 D_3 D_3,tau_L,b,tbbT,"tau_L -- (T) -- B -- (b) -- b -- [t,b]",2,1t + 2b + 1tau
456,U_3 D_3 D_3,tau,t,tbbT,"tau -- (T) -- B -- (t) -- t -- [b,b]",2,1t + 2b + 1tau
457,U_3 D_3 D_3,tau,b,tbbT,"tau -- (T) -- B -- (b) -- b -- [t,b]",2,1t + 2b + 1tau


# Introduction on syntax

### Sparticles
Notation used in for sparticles in code:
- "B",      = Bino               <br>
- "W^+"    = Charged Wino       <br>
- "W^0"    = Neutral Wino       <br>
- "G"      = Gluino             <br>
- "H^+"    = Charged Higgsino   <br>
- "H^0"    = Neutral Higgsino   <br>
- "q"      = u_L, d_L, c_L, s_L <br>
- "d"      = d_R, s_R           <br>
- "u"      = u_R, c_R           <br>
- "l"      = e_L, mu_L          <br>
- "nu"     = nu_e, nu_mu        <br>
- "e"      = e_R, mu_R          <br>
- "t_L"    = t_L                <br>
- "b_L"    = b_L                <br>
- "t"      = t_R                <br>
- "b"      = b_R                <br>
- "tau_L"  = tau_L              <br>
- "tau"     = tau_R              <br>
- "nu_tau" = nu_tau             <br>

You can use **rdef.SPARTICLES** to show all the possibilities


In [ ]:
print(rpv.rdef.SPARTICLES)

### Final State Object
Notation used in for fiinal state object in code (one-char syntax):<br>
"v" = Massive Bosons  (W,Z,H)  
"J" = jet             (u, d, c, s, t, b )         
"3" = 3rd gen jet     (t, b )         
"t" = top             (t)         
"b" = bottom jet      (b)                 
"j" = light jet       (u, d, c, s)                 
"L" = charged leptons (e, mu, tau )                     
"l" = light lepton    (e, mu )                     
"T" = tau             (tau)         
"E" = MET             (nu)    

You can use **rdef.FINAL_STATE** to show all the possibilities

In [ ]:
rpv.rdef.FINAL_STATE

# RPV Categories

**rdef.CAT_DICT** is a dictionary that contains all the catoegories in each of the RPV couplings.

In [ ]:
print(rpv.rdef.CAT_DICT["LLE"])
print(rpv.rdef.CAT_DICT["LQD"])
print(rpv.rdef.CAT_DICT["UDD"])

# Dictionaries

Note: almost every output from here on are Pandas dataframe.

### **ONE_LSP_RPV_DECAY_DICT**
It contains all information about the possibe decays of all the LSPs

In [ ]:
rpv.ONE_LSP_RPV_DECAY_DICT["LLE"]


In [ ]:
rpv.ONE_LSP_RPV_DECAY_DICT["LQD"]

In [ ]:
rpv.ONE_LSP_RPV_DECAY_DICT["UDD"]

### Understanding Signature and Chains syntax


In [ ]:
(rpv.find_one_lsp_from_signature("jjllE",rpv_coup="LLE",category="L L E").iloc[1])


#### Example Decay Chain <br>
    'W^0 -- (j) -- q -- (j) -- B -- (l) -- e -- [l,E]'
- () and [] are in signature syntax <br>
- others are in sparticle syntax <br>

So, going from the left to right, we started with a  
1. a Wino, **W^0** that decays to 1/2gen squark, **q** while producing a jet, **(j)**
2. the squark, **q** decays to a Bino, **B** while producing another jet **(j)**
3. the Bino, **B** decays to a RH 1/2gen sleptons, **e** while producing a light lepton **(l)**
4. and the sleptons, **e** finally decay to lepton and missing transverse energy  **[l,E]**

From all the signatures combined, one can see that we have:
- in our syntax: **jjllE** 
- or an easy-read syntax: **2j_l + 2l + MET**

![image](./abcrpv_alpha/example.png)

### **TWO_LSP_RPV_DECAY_DICT**
It contains all information about the possibe signatures arising from pair production of two LSPs

In [ ]:
rpv.TWO_LSP_RPV_DECAY_DICT["LLE"]

In [ ]:
rpv.TWO_LSP_RPV_DECAY_DICT["LQD"]

In [ ]:
rpv.TWO_LSP_RPV_DECAY_DICT["UDD"]

As one can see, the total amount of data can be a lot. (And it can be even more, if you generate the data yourself with different input tables. Refer paper for more details)

# Functions
To fully utilize the tables, we provide a few functions:

### 1. **find_one_lsp_from_signature(signature,rpv_coup="ALL",category="ALL",filename="",save_results=True,verbose=True)**
- Inputs:
  - **signature**    (str)  : inpput collider signature you want to know (refer syntax mentioned above)  <br> 
  - **rpv_coup**     (str)  : "LLE", "LQD", "UDD" or "ALL"(default)
  - **category**     (str)  : any category name (refer syntax mentioned above) or "ALL"(default)
  - **filename**     (str)  : file name is save_results is set to be True. (Default:  "one_lsp_that_decays_to_"+signature+"_"+rpv_coup+"_"+category.replace(" ","")+".csv")
  - **save_results** (bool) : save output into a csv file. (Default: True)

If you have a signature (eg. 2 jets(any) + 2 light leptons + MET), and you wanna know what LSP can decay to it, use: <br>


In [ ]:
rpv.find_one_lsp_from_signature("JJLLE")

There are cases where it might be too much possibilities.
You can restrict the possibilities by providing which couplings and/or which categories:

In [ ]:
rpv.find_one_lsp_from_signature("JJLLE","LQD")

In [ ]:
rpv.find_one_lsp_from_signature("JJLLE","LQD", "L_3 Q_3 D")

Similary, one can use the case for two LSPs scenario

### 2. **find_two_lsp_from_signature(signature,rpv_coup="ALL",category="ALL",filename="",save_results=True,verbose=True)**
- Inputs:
  - **signature**    (str)  : inpput collider signature you want to know (refer syntax mentioned above)  <br> 
  - **rpv_coup**     (str)  : "LLE", "LQD", "UDD" or "ALL"(default)
  - **category**     (str)  : any category name (refer syntax mentioned above) or "ALL"(default)
  - **filename**     (str)  : file name is save_results is set to be True. (Default:  "one_lsp_that_decays_to_"+signature+"_"+rpv_coup+"_"+category.replace(" ","")+".csv")
  - **save_results** (bool) : save output into a csv file. (Default: True)

If you have a signature (eg. 2 jets(any) + 3 light leptons + MET), and you wanna know which pair production of LSPs can produce such signature, use: <br>


In [ ]:
display(rpv.find_two_lsp_from_signature("JJLLLE"))


Alternatively, you can also look at what are all the possible decays that one LSP can have

### 3.**find_signatures_from_one_lsp(lsp,rpv_coup="ALL",category="ALL",filename="",save_results=True,verbose=True):**
Input
  - **lsp**          (str)  : input LSP sparticle (refer SPARTICLE syntax mentioned above) 
  - **rpv_coup**     (str)  : "LLE", "LQD", "UDD" or "ALL"(default)
  - **category**     (str)  : any category name (refer CATEGORY syntax mentioned above) or "ALL"(default)
  - **filename**     (str)  : file name is save_results is set to be True. (Default:  "one_lsp_that_decays_to_"+signature+"_"+rpv_coup+"_"+category.replace(" ","")+".csv")
  - **save_results** (bool) : save output into a csv file. (Default: True)
save_results=True,verbose=True


In [ ]:
rpv.find_signatures_from_one_lsp("G")


Similary for the two LSP scenario

### 4.**find_signatures_from_two_lsp(lspa,lspb="",rpv_coup="ALL",category="ALL",filename="",save_results=True,verbose=True):**
Input
  - **lspa**          (str)  : input LSP A sparticle (refer SPARTICLE syntax mentioned above) 
  - **lspb**          (str)  : input LSP B sparticle (refer SPARTICLE syntax mentioned above) Default: same as LSP A. Cases where there are SU(2) mass degenerate sparticles, you can include them.
  - **rpv_coup**     (str)  : "LLE", "LQD", "UDD" or "ALL"(default)
  - **category**     (str)  : any category name (refer CATEGORY syntax mentioned above) or "ALL"(default)
  - **filename**     (str)  : file name is save_results is set to be True. (Default:  "one_lsp_that_decays_to_"+signature+"_"+rpv_coup+"_"+category.replace(" ","")+".csv")
  - **save_results** (bool) : save output into a csv file. (Default: True)


In [ ]:
display(rpv.find_signatures_from_two_lsp("G"))

In [ ]:
display(rpv.find_signatures_from_two_lsp("W^0","W^+"))

# Signature Category Dictionary

### **rpv.ONE_LSP_SIG_CAT_DICT** 
categorized all possible final states of LSP decay based on their RPV categories

In [ ]:
rpv.ONE_LSP_SIG_CAT_DICT["LLE"]


In [ ]:
rpv.ONE_LSP_SIG_CAT_DICT["LQD"]


In [ ]:
rpv.ONE_LSP_SIG_CAT_DICT["UDD"]


### **rpv.TWO_LSP_SIG_CAT_DICT** 
captures all possible signature of from pair production of LSP categorized into all the categories <br>

In [ ]:
rpv.TWO_LSP_SIG_CAT_DICT["LLE"]


In [ ]:
rpv.TWO_LSP_SIG_CAT_DICT["LQD"]


In [ ]:
rpv.TWO_LSP_SIG_CAT_DICT["UDD"]


### **rpv.TWO_LSP_SIG_CAT_COMPLETE_DICT** 
Complete data used to generate **rpv.TWO_LSP_SIG_CAT_DICT**. 
Note: Computationally expensive to get proper minimal set, use the one that was produced and generate manually only if neccesary.
- LLE 2mins  <br>
- LQD 30mins<br>
- UDD 2hours <br>


In [ ]:
rpv.TWO_LSP_SIG_CAT_COMPLETE_DICT["LLE"]

# Example on how to cross check with results on our paper

In [ ]:
#Looking at Table 2 of the paper
LSP = "G"
insignature = "llllJJJJE"
incategory = "L L E"
x = rpv.find_two_lsp_from_signature(insignature,category=incategory)
display(x)
if len(x[x["LSP A"] == LSP]["Signatures"].values) > 1:
    print(rpv.get_common(x[x["LSP A"] == LSP]["Signatures"].values))
    print(rpv.minimal_sets_simplified_signatures(x[x["LSP A"] == LSP]["Signatures"].values))

# Future updates:
- Include charges of final state objects (Need to rewrite code base, in progress)
- Using all possible exact vertices instead of not_suppressed table (Need to rewrite code base, in progress)
- Incorporate code with large RPV cases (Depends on how that project goes)


# MISC Functions and Dictionaries

If you wanna get transition signature between sparticles based on your input tables:

In [ ]:
rpv.get_transition_sig("W^0","G")

If you wanna find the superset that include your input signature

In [ ]:
rpv.get_all_superset("jjlE",True,True,True)

If you wanna find all possible subsets of your signature

In [ ]:
rpv.get_subsets(["JJl","JJE"])

If you wanna convert the code signature syntax to something easier to understand:

In [ ]:
rpv.rmisc.easy_read("jjJlTTE")

If you wanna look up the input transitions tables:

In [ ]:
rpv.NOTSUP_TABLE

In [ ]:
rpv.SUP_TABLE

In [ ]:
rpv.STRSUP_TABLE

# Advanced Functions

When using different input tables, it is important to regenerate all the tables used so far.
- rpv.generate_transition_df(nv=1,save_csv=False) <br>
  - --> Default nv=1:<br>
    - when generating the table, we have used only *vertices* in the table_notsup.csv (ie transitions that are not suppressed).  <br>
    - By default, we only look for the minimal non-vanishing vertices when constructing all the tables and dictionaries. However, it is also possible to include all possible chains up to  3 *vertices* (nv=3). <br>
    - This would lead to more possible chains. And more possible signatures in most (if not all) scenario<br>
  - --> This generate main table for generation of evey other table. 
  - --> Switch on same_csv=True, to ensure results are saved. <br>
<br>
  
- rpv.generate_transitions_table()<br>
  --> generate transitions_table<br>
  
- rpv.generate_LSP_RPV_decay_table(rpv_coup):<br>
  --> generate ONE_LSP_RPV_decay_table<br>
  
- rpv.generate_LSP_sig_cat_table(rpv_coup)<br>
  --> generate ONE_LSP_sig_cat_table<br>
  
- rpv.generate_2LSP_RPV_decay_table(rpv_coup)<br>
  --> generate 2LSP_RPV_decay_table<br>
  
- rpv.generate_2LSP_sig_complete(rpv_coup)<br>
  --> generate 2LSP_sig_complete<br>
  --> **##NOT AVAILABLE YET, WORK IN PROGRESS##** computationally expensive to get proper minimal set
  

### **rpv.TRANSITION_DF**
Main table generated from inputs transition tables. All other tables are generated from this

In [ ]:
rpv.TRANSITION_DF

### **rpv.TRANSITIONS_TABLE**
Compiled Transitions tables

In [ ]:
rpv.TRANSITIONS_TABLE

### **rpv.sanity_checks()**
If one modified any of the input tables or regenerate tables with different number of vertices, it is a good thing to do some sanity checks

In [ ]:
rpv.sanity_checks()